###  神經網路(Neural  Network)是由人類的大腦神經結構的運作借鏡而來，在機器學習的世界中，神經元就像是大腦的神經細胞，是神經網路最基礎的結構，在它們相互結合下，建構整個龐大的運作網路，實現學習、處理及預測等功能。

# 使用多層感知器Multilayer perceptron ( MLP )
---
* 多層感知器就是神經網路
![MLP示意圖](MLP.jpeg)


神經元在接收輸入訊號後可以想像它是儲存了一個數字的容器，其值介於0到1之間。

以 $28 \times 28$ 像素的手寫辨識圖片來說，每個像素就是一個神經元，也就是一張圖片在輸入層總共有784個神經元，每個神經元都儲存了一個數字來代表對應像素的灰階值，數值的範圍介於0跟1之間。

而灰階值0代表黑色，1代表白色，這些數字我們稱為激勵值，數值越大則該神經元就越亮。在輸入時要將矩陣平面化(將28列前後相接成一列)，也就是這784個神經元組成了神經網路的第一層。


### Mnist資料集共有 60000 筆訓練資料，將訓練資料的  Feature(數字圖片特徵值) 和 Label(數字真值實) 都先經過預處理，作為多層感知器的輸入、輸出，然後進行模型訓練。

![MLP步驟](MLP_step.jpg)


### 模型訓練完成以後就可以用來作預測，將要預測的數字圖片，先經過預處理變成Feature(數字圖片特徵值)，就可送給模型作預測，得到 0~9 數字的預測結果。

## MLP 建立步驟

1. 資料處理
2. 建立模型
3. 訓練模型 
4. 評估模型準確度
5. 進行預測

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# 指定亂數種子
seed = 7
np.random.seed(seed)

# 載入資料集
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# 1. 資料預處理

* Features(數字影像特徵值)資料預處理: 28*28數字影像 ➔ 784 , type:float

In [ ]:
# 轉換成 2**28 = 784 的向量
X_train_list = X_train.reshape(X_train.shape[0], 28*28).astype("float32")
X_test_list = X_test.reshape(X_test.shape[0], 28*28).astype("float32")

*TEST: 可輸入  X_train.shape 查看 X_train的張量大小*

*TEST: 可輸入  X_train_list.shape 查看 X_train_list的張量大小*

* Labels(數字影像真實值)資料預處理: 數字影像Image的數字影像標準化

In [ ]:
# 因為是固定範圍, 所以執行標準化, 從 0-255 至 0-1
X_train_normalize = X_train_list / 255
X_test_normalize  = X_test_list / 255

Y_test_bk = Y_test.copy() 

### labels資料預處理

* label原本是0~9的數字  ➔  10個0或1的組合
* 數字3經過 one-hot encoding ➔ 0001000000

In [ ]:
# One-hot編碼 (輸出對應)
Y_train_OneHot = to_categorical(Y_train)
Y_test_OneHot = to_categorical(Y_test)

*TEST: 查看結果*

In [ ]:
print (Y_train[0] )
print (Y_train_OneHot[0] )

### 啟動函數 Activation Function

* Step function
* Sigmoid function 
* ReLu (Rectified Linear Unit) function
* Tanh function
* Softmax function

# 2.建立模型

In [ ]:
# 建立 Sequential 順序模組
model = Sequential()


# 模型加入【輸入層】與【隱藏層】
model.add( Dense( input_dim = 28*28             # 輸入層有 28*28=784 個神經元
                , units = 256                   # 隱藏層有 256 個神經元    (值越大, 訓練越精準, 相對訓練時間也越久)
                , kernel_initializer = 'normal' # 使用 normal 初始化 weight 權重與 bias 偏差值
                , activation = 'relu'           # 使用 relu 啟動函數
                ))


# 模型加入【輸出層】
model.add( Dense( units = 10                    # 輸出層有 10 個神經元 (因為數字只有 0 ~ 9)
                , kernel_initializer = 'normal' # 使用 normal 初始化 weight 權重與 bias 偏差值
                , activation = 'softmax'        # 使用 softmax 啟動函函數 (softmax 值越高, 代表機率越大)
                ))

# 顯示模型摘要資訊
print (model.summary() )   

para # : 784*256+256 = 200960

* $h_1=relu( X \times W_1 +b_1 )$ 

para # : 256*10+ 10 = 2570

* $y=softmax( h_2 \times W_2 +b_2 )$ 

### 損失函數 loss Function

* 均方誤差 mean square error
* 交叉熵 Croee-Entropy

# 3. Model 模型進行【Train 訓練】

In [ ]:
## 設定模型的訓練方式 ##

model.compile( loss='categorical_crossentropy' # 設定 Loss 損失函數 為 categorical_crossentropy 使用交叉熵
             , optimizer = 'adam'              # 設定 Optimizer 最佳化方法 為 adam
             , metrics = ['accuracy']          # 設定 Model 評估準確率方法 為 accuracy
             )

In [ ]:
# 訓練模型
history = model.fit(               # 訓練的歷史記錄, 會會回傳到指定變數 history
          x = X_train_normalize    # 設定 圖片 Features 特徵值 (mnist 提供 60000 筆資料)
        , y = Y_train_OneHot       # 設定 圖片 Label    真實值 (mnist 提供 60000 筆資料)
        , validation_split = 0.2   # 設定 有多少筆驗證         (60000*0.2=12000 筆驗證, 60000*0.8=48000 筆訓練)
        , epochs = 10              # 設定 訓練次數             (值 10 以上,  值越大, 訓練時間越久, 但訓練越精準)
        , batch_size = 128         # 設定 訓練時每批次有多少筆 (值 100 以上, 值越大, 訓練速度越快, 但需記憶體要夠大)
        , verbose = 2              # 是否 顯示訓練過程         (0: 不顯示, 1: 詳細顯示, 2: 簡易顯示)
)

# 執行的顯示結果 (這會花一些時間, 然後會逐次顯示訓練結果)
# loss:     使用訓練資料, 得到的損失函數誤差值 (值越小, 代表準確率越高)
# acc:      使用訓練資料, 得到的評估準確率    (值在 0~1, 值越大, 代表準確率越高)
# val_loss: 使用驗證資料, 得到的損失函數誤差值 (值越小, 代表準確率越高)
# val_acc:  使用驗證資料, 得到的評估準確率    (值在 0~1, 值越大, 代表準確率越高)

#  4. 評估模型準確度
**顯示圖表來分析模型的訓練過程** 

In [ ]:
# 顯示圖表來分析模型的訓練過程
import matplotlib.pyplot as plt


# 顯示訓練和驗證損失
loss = history.history["loss"]
epochs = range(1, len(loss)+1)
val_loss = history.history["val_loss"]

plt.plot(epochs, loss, "bo-", label="Training Loss")
plt.plot(epochs, val_loss, "ro--", label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()



# 顯示訓練和驗證準確度
acc = history.history['acc']
epochs = range(1, len(acc)+1)
val_acc = history.history['val_acc']

plt.plot(epochs, acc, "bo-", label="Training Acc")
plt.plot(epochs, val_acc, "ro--", label="Validation Acc")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# 儲存模型
model.save('mnist_mlp_h256.h5')

**overfitting 過度擬合**

# 5. 進行預測

* 評估模型準確度

In [ ]:
sores = model.evaluate( X_test_normalize , Y_test_OneHot)
print ()
print ( 'accuracy=',sores[1] )

* 預測結果與原始圖形

In [ ]:
## 執行預測
Y_pred = model.predict_classes(X_test_normalize)

In [ ]:
Y_pred

In [ ]:
import  matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))

for i in range(20):
  ax = plt.subplot(4,5,i+1)
  ax.imshow(X_test[i] ,cmap='gray')
  ax.set_title('Label:'+str(Y_test[i])+ '  pre:'+str(Y_pred[i]) ,fontsize= 11      )
  ax.axis('off')
plt.subplots_adjust( hspace=0.5)
plt.show()

### 計算分類的預測值

**混淆矩陣**
* 混淆矩陣(Confusion matrix)是一種對分類模型進行效果評估的方法
* 通過將模型預測的數據與測試數據進行對比，使用準確率，覆蓋率和命中率等指標對模型的分類效果進行度量。 

In [ ]:
# 計算分類的預測值
print("\nPredicting ...")
Y_pred = model.predict_classes(X_test_normalize)


import pandas as pd
# 顯示混淆矩陣
tb = pd.crosstab(Y_test_bk.astype(int), Y_pred.astype(int),
                 rownames=["label"], colnames=["predict"])
print(tb)

**建立真實值與預測 dataframe**

In [ ]:
df = pd.DataFrame( {'label': Y_test_bk.astype(int), 'predict': Y_pred.astype(int) }   )
df[:10]

**匯出0~9數字預測機率**

In [ ]:
i = 7
digit = X_test[i].reshape(28, 28)
# 將圖片轉換成 4D 張量
X_test_digit = X_test[i].reshape(1, 28, 28, 1).astype("float32")

print ( '-------')
# 因為是固定範圍, 所以執行正規化, 從 0-255 至 0-1
X_test_digit = X_test_digit / 255

# 繪出圖表的預測結果
plt.figure()
plt.subplot(1,2,1)
plt.title("Example of Digit:" + str(Y_test[i]))
plt.imshow(digit, cmap="gray")
plt.axis("off")

# 預測結果的機率
print("Predicting ...")
probs = model.predict_proba(X_test_normalize)
plt.subplot(1,2,2)
print (probs.shape)
plt.title("Probabilities for Each Digit Class")
plt.bar(np.arange(10), probs[i], align="center")
plt.xticks(np.arange(10),np.arange(10).astype(str))
plt.show()

In [ ]:
df[ ( df.label==5  ) & (df.predict==3)   ]

In [ ]:
index = 340

import  matplotlib.pyplot as plt

# 繪出圖表的預測結果
plt.figure()
plt.subplot(1,2,1)

plt.imshow(X_test[index] ,cmap='gray')
plt.title('Label:'+str(Y_test[index])+ '  pre:'+str(Y_pred[index]) ,fontsize= 12      )
plt.axis('off')

# 預測結果的機率
print("Predicting ...")
probs = model.predict_proba(X_test_normalize)
plt.subplot(1,2,2)
print (probs.shape)
plt.title("Probabilities for Each Digit Class")
plt.bar(np.arange(10), probs[index], align="center")
plt.xticks(np.arange(10),np.arange(10).astype(str))
plt.show()

### Overfitting問題
* Overfitting(過度訓練):當可選擇的參數自由度超過資料所包含的資訊內容時，這會破壞模型一般化的能力。
* 解決方法:

  (1)增加數據量, 大部分過擬合產生的原因是因為數據量太少了。
  
  (2)加入DropOut功能，在訓練的時候，我們隨機忽略掉一些神經元和神經元的連結，讓每一次預測結果都不會太過依賴於其中某部分特定神經元。